In [1]:
from utils import *

In [2]:
import lutorpy as lua
require("nn")
require("optim")
require("cutorch")
require("cunn")

True

In [3]:
np.random.seed(0)

In [4]:
M = M_star = 5
N = NUMBER_OF_PEDESTRIANS
X = Y = 256

In [5]:
location = np.load('src/pedestrians_location.npy') # [NUMBER_OF_PEDESTRIANS x NUMBER_OF_FRAMES x 2]
location[:,:,0] = (location[:,:,0] / X).astype(np.int64)
location[:,:,1] = (location[:,:,1] / Y).astype(np.int64)
# location[i,t] = [x,y] -- location of i-th pedestrian at time point t (aka frame 20*t) -- [0,0] for absent pedestians

paths = np.load('src/all_not_ext_paths_with_ids.npy') # paths[j] = [i,t,x,y] -- all pedestrians' coordinates at all frames
paths[:,2] = (paths[:,2] / X).astype(np.int64)
paths[:,3] = (paths[:,3] / Y).astype(np.int64)

In [6]:
# M-1 time points before curr_time and M_star after
def get_t(curr_time, M, M_star):
    step = 20
    return np.arange(curr_time - step*(M-1), curr_time + step*(M_star+1), step)

In [7]:
t = get_t(100, M, M_star)
ind_t = (t/20).astype(int) # for indexing in location
t

array([ 20,  40,  60,  80, 100, 120, 140, 160, 180, 200])

In [8]:
pedestrians_in_scene = paths[paths[:,1] == t[M-1]] # getting all pedestrians who located on map at curr_time

disp_volume = np.zeros((X, Y, 2*M))

for p in pedestrians_in_scene:
    l = location[p[0],ind_t[:M]] # locations of pedestrian at t_1, t_2, ..., t_M time points

    # filling zeros absent pedestrians
    for i, row in enumerate(l):
        if (row == 0).all():
            l[i] = l[-1] + np.array([X, Y]) # to get zero after next code line
            
    d = (l[-1] - l) / np.array([X, Y]) + 1
    disp_volume[p[2], p[3]] = d.flatten() # put in [X, Y]-th cell of disp_volume displacement vector

In [ ]:
# THERE IS A PROBLEM HERE:
# we lose pedestrians' ids after coding into disp_volume so to decode we need somehow match previous path and predicted

# displacement volume decode
xs, ys = disp_volume[:,:,0].nonzero() # getting indices of non-zero predicted coordinates at first (t_M+1) timepoint
for i, j in zip(xs,ys):
    np.array([i,j]) - (disp_volume[i,j].reshape(-1,2) - 1) * np.array([X, Y])

In [9]:
model = nn.Sequential()

filters_num = 64
kW = kH = 3
padW = padH = 1

# https://github.com/torch/nn/blob/master/doc/convolution.md#nn.SpatialConvolution
model._add(nn.SpatialConvolution(2*M, filters_num, kW, kH, 1, 1, padW, padH)) # conv1
model._add(nn.SpatialConvolution(filters_num, filters_num, kW, kH, 1, 1, padW, padH)) # conv2
model._add(nn.SpatialConvolution(filters_num, filters_num, kW, kH, 1, 1, padW, padH)) # conv3

# https://github.com/torch/nn/blob/master/doc/convolution.md#nn.SpatialMaxPooling
model._add(nn.SpatialMaxPooling(2, 2, 2, 2))

# https://github.com/torch/nn/blob/master/doc/simple.md#nn.CAdd
model._add(nn.CAdd(1, int(Y/2), int(X/2))) # bias

model._add(nn.SpatialConvolution(filters_num, filters_num, kW, kH, 1, 1, padW, padH)) # conv4
model._add(nn.SpatialConvolution(filters_num, filters_num, kW, kH, 1, 1, padW, padH)) # conv5
model._add(nn.SpatialConvolution(filters_num, 2*M_star, kW, kH, 1, 1, padW, padH)) # conv6

# https://github.com/torch/nn/blob/master/doc/convolution.md#spatialfullconvolution
model._add(nn.SpatialFullConvolution(2*M_star, 2*M_star, 4, 4, 2, 2)) # deconv

<Lua table at 0x41cf09b0>

In [ ]:
model._forward(torch.fromNumpyArray(disp_volume.T))